In [154]:
import pandas as pd

train_df = pd.read_csv("training data.csv")
train_df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Malta,2008,Developed,80.0,64.0,0,7.14,2655.573684,86.0,1,...,72.0,8.15,72.0,0.1,21928.767000,49379.0,0.7,0.7,0.813,14.6
1,Congo,2005,Developing,55.3,394.0,8,2.03,0.000000,NaN,146,...,62.0,2.42,62.0,5.9,NaN,NaN,8.8,8.5,0.496,9.4
2,Burkina Faso,2009,Developing,56.9,283.0,44,4.55,81.143047,92.0,54118,...,91.0,7.41,92.0,1.1,552.745552,1514199.0,9.3,8.8,0.356,5.9
3,Guinea-Bissau,2011,Developing,57.1,289.0,4,3.57,40.453674,86.0,0,...,85.0,5.46,86.0,5.7,692.699890,1596154.0,7.8,7.7,0.410,9.0
4,Myanmar,2007,Developing,64.5,217.0,58,0.26,0.530573,85.0,1088,...,84.0,1.68,86.0,0.6,41.451000,49171586.0,13.2,13.5,0.484,8.1


In [155]:
train_df = train_df.drop(["Country"], axis=1)
train_df.dropna(subset=['Life expectancy'], inplace=True)

In [156]:
train_df = pd.get_dummies(train_df, columns=["Status"])

In [157]:
X = train_df.drop(["Life expectancy"], axis=1)
y = train_df[["Life expectancy"]].to_numpy().reshape((-1,))

In [158]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=5)
X_imputed = knn_imputer.fit_transform(X)

In [159]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import lightgbm as lgb
import warnings
import sys, os

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
    
poly_ker = PolynomialFeatures(degree=1)
X_pol = poly_ker.fit_transform(X_imputed)
# X_test_pol = poly_ker.transform(X_test)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_pol)
# X_test_scaled = scaler.transform(X_test_pol)

# Convert scaled data back to a DataFrame
X_scaled = pd.DataFrame(X_scaled)

In [160]:
LGBModel = lgb.LGBMRegressor()
LGBModel.fit(X_scaled, y)
X_new = X_scaled.copy()
X_new['Hardness_pred'] = LGBModel.predict(X_scaled)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000346 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3402
[LightGBM] [Info] Number of data points in the train set: 2050, number of used features: 21
[LightGBM] [Info] Start training from score 69.395610


In [161]:
lin_reg = LinearRegression()
lin_reg.fit(X_scaled, y)
lin_reg.score(X_scaled, y)

0.826810576701616

In [162]:
# # Ridge Regression GridSearchCV with Polynomial features
# ridge_poly_pipeline = make_pipeline(PolynomialFeatures(), StandardScaler(), Ridge())
# ridge_poly_params = {
#     # 'polynomialfeatures__degree': [1, 2, 3],  # Example degree values for Polynomial features
#     'ridge__alpha': [1, 0.1, 10.0, 20, 50]
# }
# ridge_poly_grid = GridSearchCV(ridge_poly_pipeline, param_grid=ridge_poly_params, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
# ridge_poly_grid.fit(X_train, y_train)

# # Lasso Regression GridSearchCV with Polynomial features
# lasso_poly_pipeline = make_pipeline(PolynomialFeatures(), StandardScaler(), Lasso())
# lasso_poly_params = {
#     # 'polynomialfeatures__degree': [2, 3, 4],  # Example degree values for Polynomial features
#     'lasso__alpha': [1e-2, 1e-3, 1e-4, 1e-5]
# }
# lasso_poly_grid = GridSearchCV(lasso_poly_pipeline, param_grid=lasso_poly_params, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
# lasso_poly_grid.fit(X_train, y_train)

In [163]:
# # LightGBM Regression GridSearchCV
# lgb_pipeline = make_pipeline(PolynomialFeatures(), StandardScaler(), lgb.LGBMRegressor(verbose=-1))
# lgb_params = {
#     'lgbmregressor__n_estimators': [100, 200],
#     'lgbmregressor__learning_rate': [0.05, 0.1],
#     'lgbmregressor__max_depth': [3, 4]
# }
# lgb_grid = GridSearchCV(lgb_pipeline, param_grid=lgb_params, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
# lgb_grid.fit(X_train, y_train)

In [164]:
# # Random Forest Regression GridSearchCV
# rnf_pipeline = make_pipeline(PolynomialFeatures(), StandardScaler(), RandomForestRegressor())
# rnf_params = {
#     'randomforestregressor__n_estimators': [300, 400],
#     'randomforestregressor__max_depth': [15, 20]
# }
# rnf_grid = GridSearchCV(rnf_pipeline, param_grid=rnf_params, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1)
# rnf_grid.fit(X_train, y_train)

In [165]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, InputLayer
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp

In [166]:
from tensorflow.keras.callbacks import Callback

# Define a custom callback
class PrintEpochLoss(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 50 == 0:
            print(f"Epoch {epoch} - Loss: {logs['loss']}")

In [167]:
def loss_fn(y_true, y_pred):
    return tfp.stats.windowed_mean(tf.abs(y_true - y_pred))
def metric_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=100) - tfp.stats.percentile(tf.abs(y_true - y_pred), q=0)

In [168]:
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200, verbose=2, mode='min', restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00001),
    tf.keras.callbacks.TerminateOnNaN(),
    PrintEpochLoss()
] 

In [169]:
# Define the neural network model
nn_model = Sequential([
    InputLayer(input_shape=(X_new.shape[1],)),
    BatchNormalization(epsilon=0.00001),
    Dense(64, activation='sigmoid'),
    Dense(64, activation='sigmoid'),
    Dense(32, activation='sigmoid'),
    Dense(1)  # Single output node for regression
])

# Compile the model with Mean Absolute Error (MAE) as the metric
nn_model.compile(optimizer=Adam(0.013, beta_1=0.5), loss=loss_fn)

In [170]:
print_epoch_loss = PrintEpochLoss()

# Train the model on the training data
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_new, y, test_size=0.2, random_state=42)

# Train the model on the training data with the defined callback
history = nn_model.fit(
    X_train_val, y_train_val, 
    epochs=500,
    batch_size=32,
    class_weight=LGBModel.class_weight,
    validation_data=(X_test_val, y_test_val), 
    callbacks=callbacks_list,
    verbose=0  # Set verbose to 0 to avoid redundant output
)

Epoch 0 - Loss: 57.92087936401367
Epoch 50 - Loss: 1.3829365968704224
Epoch 100 - Loss: 1.4581743478775024
Epoch 150 - Loss: 1.4402942657470703
Epoch 200 - Loss: 1.4215245246887207
Epoch 250 - Loss: 1.3654329776763916
Epoch 300 - Loss: 1.4613370895385742
Restoring model weights from the end of the best epoch: 138.
Epoch 338: early stopping


In [171]:
# Evaluate the model on the test data
mae = nn_model.evaluate(X_test_val, y_test_val, verbose=0)
print(f"MAE on test data: {mae}")

# Predictions on test data
y_pred = nn_model.predict(X_new)

# Calculate Mean Absolute Error
mae_score = mean_absolute_error(y, y_pred)
print(f"Mean Absolute Error: {mae_score}")

MAE on test data: 0.8637043237686157
65/65 [==============================] - 0s 1ms/step
Mean Absolute Error: 0.8465575680616426


In [172]:
# # Evaluate the best models
# ridge_poly_best = ridge_poly_grid.best_estimator_
# lasso_poly_best = lasso_poly_grid.best_estimator_
# # gb_poly_best = gb_grid.best_estimator_
# # lgb_best = lgb_grid.best_estimator_
# rnf_best = rnf_grid.best_estimator_

# # Evaluate on test data
# ridge_poly_score = ridge_poly_best.score(X_test, y_test)
# lasso_poly_score = lasso_poly_best.score(X_test, y_test)
# # gb_poly_score = gb_poly_best.score(X_test, y_test)
# # lgb_score = lgb_best.score(X_test, y_test)
# rnf_score = rnf_best.score(X_test, y_test)

# # Print the best parameters and score
# print("Best Parameters for Ridge Regression:", ridge_poly_grid.best_params_)
# print("Ridge Regression Score:", ridge_poly_score)

# # Print the best parameters and score
# print("Best Parameters for Lasso Regression:", lasso_poly_grid.best_params_)
# print("Lasso Regression Score:", lasso_poly_score)

# # # Print the best parameters and score
# # print("Best Parameters for Gradient Boosting:", gb_grid.best_params_)
# # print(f'Gradient Boosting Score: {gb_score}')

# # # Print LightGBM score 
# # print("Best Parameters for LightGBM:", lgb_grid.best_params_)
# # print(f'LightGBM Score: {lgb_score}')

# # Print Random Forest Regression score 
# print("Best Parameters for Random Forest Regression:", rnf_grid.best_params_)
# print(f'Random Forest Regression Score: {rnf_score}')

In [173]:
test_df = pd.read_csv("testing data.csv")

In [174]:
X_test = test_df.drop(["Country", "Row_id"], axis=1)
X_test = pd.get_dummies(X_test, columns=["Status"])
X_test_imputed = knn_imputer.transform(X_test)

In [175]:
X_test = poly_ker.transform(X_test_imputed)
X_test = scaler.transform(X_test)

# Convert scaled data back to a DataFrame
X_test = pd.DataFrame(X_test)

In [176]:
X_test_new = X_test.copy()
X_test_new['Hardness_pred'] = LGBModel.predict(X_test)

In [177]:
y_test = nn_model.predict(X_test_new)

28/28 [==============================] - 0s 1ms/step


In [178]:
# Create csv of y_test with columns "id" and "Hardness"
y_test_df = pd.DataFrame({'Row_id': test_df['Row_id'], 'Life expectancy ': y_test.reshape((-1,))})
y_test_df.to_csv('predictions.csv', index=False)